# Netflix Movie Recommender

#### import relevant dependancies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import missingno as msno
import autoreload
import warnings
import os
import csv
import sys

from collections import defaultdict, Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

%matplotlib inline

sb.set()
sb.set_style('darkgrid')
sb.set_palette('viridis')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r"D:\Open Classroom\Datasets\Netflix\netflix_titles.csv")
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [3]:
df.shape

(6234, 12)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 12 columns):
show_id         6234 non-null int64
type            6234 non-null object
title           6234 non-null object
director        4265 non-null object
cast            5664 non-null object
country         5758 non-null object
date_added      6223 non-null object
release_year    6234 non-null int64
rating          6224 non-null object
duration        6234 non-null object
listed_in       6234 non-null object
description     6234 non-null object
dtypes: int64(2), object(10)
memory usage: 584.6+ KB


In [5]:
df.isnull().sum()

show_id            0
type               0
title              0
director        1969
cast             570
country          476
date_added        11
release_year       0
rating            10
duration           0
listed_in          0
description        0
dtype: int64

In [6]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


### Feature Engineering

In [7]:
tf_vect = TfidfVectorizer(min_df=3, max_features=None, ngram_range=(1,3), 
                          stop_words='english', strip_accents='unicode')

df['description'] = df['description'].fillna('')

desc_matric = tf_vect.fit_transform(df['description'])

In [8]:
desc_matric

<6234x7314 sparse matrix of type '<class 'numpy.float64'>'
	with 80962 stored elements in Compressed Sparse Row format>

In [9]:
desc_matric.shape

(6234, 7314)

In [10]:
sig = sigmoid_kernel(desc_matric, desc_matric)



In [11]:
sig[0]

array([0.76165157, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
       0.76159416])

In [12]:
indices = pd.Series(df.index, df['title']).drop_duplicates()
indices

title
Norm of the North: King Sized Adventure           0
Jandino: Whatever it Takes                        1
Transformers Prime                                2
Transformers: Robots in Disguise                  3
#realityhigh                                      4
                                               ... 
Red vs. Blue                                   6229
Maron                                          6230
Little Baby Bum: Nursery Rhyme Friends         6231
A Young Doctor's Notebook and Other Stories    6232
Friends                                        6233
Length: 6234, dtype: int64

In [13]:
indices['Red vs. Blue']

6229

In [14]:
sig[6229]

array([0.76159416, 0.76159416, 0.76159416, ..., 0.76159416, 0.76159416,
       0.76159518])

In [15]:
list(enumerate(sig[indices['Red vs. Blue']]))[:10]

[(0, 0.7615941559557649),
 (1, 0.7615941559557649),
 (2, 0.7615941559557649),
 (3, 0.7615941559557649),
 (4, 0.7615941559557649),
 (5, 0.7615952284831071),
 (6, 0.7615941559557649),
 (7, 0.7615941559557649),
 (8, 0.7615941559557649),
 (9, 0.7615941559557649)]

In [16]:
sorted(list(enumerate(sig[indices['Red vs. Blue']])))[:10]

[(0, 0.7615941559557649),
 (1, 0.7615941559557649),
 (2, 0.7615941559557649),
 (3, 0.7615941559557649),
 (4, 0.7615941559557649),
 (5, 0.7615952284831071),
 (6, 0.7615941559557649),
 (7, 0.7615941559557649),
 (8, 0.7615941559557649),
 (9, 0.7615941559557649)]

In [17]:

def netflix_recommender(title, sig=sig):
    
    db_mov_list = [x for x in df['title']]
    
    if title in db_mov_list:
    
        idx = indices[title]

        mov_list = list(enumerate(sig[idx]))

        sort_mov = sorted(mov_list, key = lambda x: x[1], reverse = True)

        top_ten = sort_mov[0:10]

        movies_rec = [x[0] for x in top_ten]
        
        print("")
        
        print('We Recommend you watch the following ')

        return df[['title', 'type', 'listed_in', 'duration', 'release_year']].iloc[movies_rec].sort_values(by = ["release_year", "duration"], ascending = False)
    
    else:
        print('__DATABASE ERROR___ During handling of the above exception, another exception occurred: InvalidIndexError(key)')
        print('MOVIE NOT FOUND')

In [18]:
netflix_recommender(input('Movie Watched: ').lower().title())

Movie Watched:  El Cartel



We Recommend you watch the following 


,title,type,listed_in,duration,release_year
5719,The End of the F***ing World,TV Show,"British TV Shows, International TV Shows, TV C...",2 Seasons,2019
4960,Diagnosis,TV Show,"Docuseries, Science & Nature TV",1 Season,2019
1802,In Family I Trust,Movie,"Comedies, Dramas, International Movies",98 min,2018
5797,She's Gotta Have It,TV Show,"Romantic TV Shows, TV Comedies, TV Dramas",2 Seasons,2018
244,Larceny,Movie,Action & Adventure,87 min,2017
5833,Father Brown,TV Show,"British TV Shows, Crime TV Shows, TV Dramas",6 Seasons,2017
4798,Undercover Law,TV Show,"Crime TV Shows, International TV Shows, Spanis...",1 Season,2017
5,Apaches,TV Show,"Crime TV Shows, International TV Shows, Spanis...",1 Season,2016
1996,Dandy,TV Show,"Crime TV Shows, International TV Shows, Spanis...",1 Season,2016
4856,El Cartel,TV Show,"Crime TV Shows, International TV Shows, Spanis...",1 Season,2008


In [26]:
import plotly.graph_objects as go

l=netflix_recommender('El Cartel', sig)
fig = go.Figure(data=[go.Table(header=dict(values=l,fill_color='orange'))
                     ])
fig.show()


We Recommend you watch the following 


IndexError: positional indexers are out-of-bounds